In [ ]:
#pwatson 2018
#this code is based on the excellent guide by Dr. Jason Brownlee from machinelearningmastery.com
#you should go there to learn all kinds of handy dandy DL stuff!

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [31]:
#for i in $(wget -q -O - http://www.hplovecraft.com/writings/fiction/ | grep "^<li>" | grep aspx | grep -o -E '\w\.aspx' | head -1); do wget http://www.hplovecraft.com/writings/texts/fiction/"$i"; done


SyntaxError: invalid syntax (<ipython-input-31-babcaa8daf61>, line 1)

In [12]:
# load ascii text and covert to lowercase
filename = "lovecraft.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("CharSet: ", chars)

Total Characters:  2643141
Total Vocab:  63
CharSet:  ['\n', ' ', '!', '"', '#', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', '[', '\\', ']', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '°', '—', '•', '■']


In [6]:
#find character freqency and set filtering criteria
char_freq={}
for i in chars:
    char_freq[i]=raw_text.count(i)/len(raw_text)

import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_char_freq = sorted(char_freq.items(), key=operator.itemgetter(1))

print(sorted_char_freq)

[('#', 3.7833774285972637e-07), (']', 7.566754857194527e-07), ('>', 7.566754857194527e-07), ('■', 1.1350132285791792e-06), ('[', 1.5133509714389055e-06), ('{', 3.4050396857375373e-06), ('&', 4.16171517145699e-06), ('•', 5.675066142895895e-06), ('°', 1.3241821000090422e-05), ('\\', 2.1565251343004404e-05), ('/', 2.8753668457339205e-05), ('<', 3.178037040021701e-05), ('^', 3.707709880025319e-05), ('4', 3.934712525741154e-05), (':', 5.637232368609923e-05), ('3', 6.507409177187293e-05), ('6', 6.658744274331184e-05), ('(', 6.696578048617156e-05), (')', 7.03708201719091e-05), ('5', 7.264084662906747e-05), ('8', 7.793757502910363e-05), ('7', 8.890936957203569e-05), ('9', 0.00011501467382935682), ('2', 0.0001248514551437097), ('0', 0.00015284844811532945), ('!', 0.0001536051236010489), ('?', 0.00018765552045842427), ('1', 0.0002988868168591838), ('"', 0.0005395096213179698), ('z', 0.0006129071434327567), ('j', 0.0006768462219760504), ('q', 0.0008542866233772621), ('—', 0.0011618752083222196), 

In [24]:
dead_chars=["#",">","■","[","]","{","}","•","°","\\","/","<","^",]
cooked_text=raw_text
for i in dead_chars:
    cooked_text=cooked_text.replace(i,"")

chars = sorted(list(set(cooked_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(cooked_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("CharSet: ", chars)

Total Characters:  2642755
Total Vocab:  51
CharSet:  ['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—']


In [21]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = cooked_text[i:i + seq_length]
    seq_out = cooked_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Patterns:  2642655


In [30]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [23]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list, shuffle=False)

Epoch 1/20
  12800/2642655 [..............................] - ETA: 2:54:35 - loss: 3.0698

KeyboardInterrupt: 

In [ ]:
# load the network weights
filename = filepath
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."